## Spotipy

In [4]:
import spotify_lyrics_scraper as spotify

# Ottieni il token SP_DC
token = spotify.getToken("AQBSz33_ODmldHiskIdlUiuu4yS92-tR4swS5FeqJuHYfBNUdEs3j0mWK4Hmo9wDc3eTK6IRAE0xFoXnH94CkXHKUWvZuFLTdTxyBdOhLCtSyhZLRWM21tgwJDAI1MxFAA_dEPWMtVMxGd9GefWoL4JbtxCXlsfKvk0CF5zpBQ1m_JskXi7hHLj-wLJBlNdTlpdkeA_LZwBCZG5yQowbiE4hO8K5")

# Ottieni i testi della canzone
lyrics_data = spotify.getLyrics(token, songName="Paradise city")

# Controlla se la richiesta è andata a buon fine
if lyrics_data['status']:
    # Estrai le linee di testo
    lyrics_lines = lyrics_data['message']['lyrics']['lines']
    
    # Stampa solo il testo della canzone
    for line in lyrics_lines:
        print(line['words'])
else:
    print("Errore nel recupero dei testi.")


Take me down to the Paradise City
Where the grass is green and the girls are pretty
Take me home (I want you, please, take me home)
Take me down to the Paradise City
Where the grass is green and the girls are pretty
Take me home (I want you, please, take me home)
♪
Just an urchin living under the street
I'm a hard case that's tough to beat
I'm your charity case so buy me somethin' to eat
I'll pay you at another time
Take it to the end of the line
Rags and riches, or so they say, you gotta
Keep pushing for the fortune and fame
You know it's, it's all a gamble when it's just a game
You treat it like a capital crime
Everybody's doing their time
Take me down to the Paradise City
Where the grass is green and the girls are pretty
Oh, won't you please take me home? Yeah-yeah
Take me down to the Paradise City
Where the grass is green and the girls are pretty
Take me home
Strapped in the chair of the city's gas chamber
Why I'm here, I can't quite remember
The surgeon general says it's hazardous

In [5]:
# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the pre-trained model and tokenizer
model_name = "j-hartmann/emotion-english-distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Combine all the lyrics lines into a single string
song_lyrics = "\n".join([line['words'] for line in lyrics_lines])

# Tokenize the input text
inputs = tokenizer(song_lyrics, return_tensors="pt", truncation=True, padding=True)

# Get the model outputs
with torch.no_grad():
    outputs = model(**inputs)

# Extract logits and compute probabilities
logits = outputs.logits
probabilities = torch.nn.functional.softmax(logits, dim=1)[0]

# Define the emotions
emotions = [
    'admiration',
    'amusement',
    'anger',
    'love',
    'pain',
    'caring',
    'confusion'
]

# Map emotions to probabilities
emotion_probs = {emotion: float(probabilities[idx]) for idx, emotion in enumerate(emotions)}

# Sort emotions by probability in descending order
sorted_emotions = sorted(emotion_probs.items(), key=lambda x: x[1], reverse=True)

# Print the emotions and their corresponding probabilities
print("Emotion Classification:\n")
for emotion, prob in sorted_emotions:
    print(f"{emotion.capitalize()}: {prob:.4f}")


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Emotion Classification:

Pain: 0.3304
Anger: 0.1785
Caring: 0.1659
Admiration: 0.1429
Amusement: 0.0889
Confusion: 0.0545
Love: 0.0389


## Lexicon

In [8]:
# Import necessary libraries
import spotify_lyrics_scraper as spotify
import nltk
from nltk.tokenize import word_tokenize
from collections import defaultdict
import requests


# Function to load the NRC Emotion Lexicon
def load_nrc_lexicon(local_path='NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'):
    emotion_dict = defaultdict(set)
    with open(local_path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) == 3:
                word, emotion, association = parts
                if int(association) > 0:
                    emotion_dict[emotion].add(word)
    return emotion_dict

# Obtain the SP_DC token
token = spotify.getToken("AQBSz33_ODmldHiskIdlUiuu4yS92-tR4swS5FeqJuHYfBNUdEs3j0mWK4Hmo9wDc3eTK6IRAE0xFoXnH94CkXHKUWvZuFLTdTxyBdOhLCtSyhZLRWM21tgwJDAI1MxFAA_dEPWMtVMxGd9GefWoL4JbtxCXlsfKvk0CF5zpBQ1m_JskXi7hHLj-wLJBlNdTlpdkeA_LZwBCZG5yQowbiE4hO8K5")
# Replace with your actual token

# Obtain the lyrics of the song
lyrics_data = spotify.getLyrics(token, songName="Dancing Queen")

# Check if the request was successful
if lyrics_data['status']:
    # Extract the lyrics lines
    lyrics_lines = lyrics_data['message']['lyrics']['lines']
    
    # Combine the lyrics into a single string
    song_lyrics = '\n'.join([line['words'] for line in lyrics_lines])
    
    # Tokenize the song lyrics
    tokens = word_tokenize(song_lyrics.lower())
    
    # Load the lexicon
    emotion_lexicon = load_nrc_lexicon()
    
    # Count the emotions
    emotion_counts = defaultdict(int)
    for token in tokens:
        for emotion, words in emotion_lexicon.items():
            if token in words:
                emotion_counts[emotion] += 1
    
    # Total words matched
    total_emotion_words = sum(emotion_counts.values())
    
    # Calculate emotion percentages
    emotion_percentages = {emotion: (count / total_emotion_words) if total_emotion_words > 0 else 0 for emotion, count in emotion_counts.items()}
    
    # Sort emotions by percentage
    sorted_emotions = sorted(emotion_percentages.items(), key=lambda x: x[1], reverse=True)
    
    # Print the results
    print("Emotion Classification:\n")
    for emotion, percentage in sorted_emotions:
        print(f"{emotion.capitalize()}: {percentage:.4f}")
else:
    print("Errore nel recupero dei testi.")


Emotion Classification:

Positive: 0.2419
Joy: 0.2097
Anticipation: 0.1774
Surprise: 0.1290
Trust: 0.1129
Sadness: 0.0645
Fear: 0.0484
Negative: 0.0161


## Facial Recognition with DLIB

In [12]:
import dlib
import cv2
import numpy as np
from imutils import face_utils

# Load dlib's face detector and face recognition model
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')  # Download this model file from dlib
face_rec_model = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')  # Download this too

# List to store face encodings and corresponding labels (names)
known_face_encodings = []
known_face_labels = []

# Load and encode your database of faces (use your database here)
def load_known_faces():
    # Example: You have images and labels in a folder (replace with your actual data loading logic)
    # For each face image, detect the face, encode it, and store the encoding with the label (name)
    images = ['/Users/vittoriomocchi/Documents/Vitos/Projects/Emotion-Song-Recommender-System-main/database/Elena.jpg', '/Users/vittoriomocchi/Documents/Vitos/Projects/Emotion-Song-Recommender-System-main/database/Lena.jpg', '/Users/vittoriomocchi/Documents/Vitos/Projects/Emotion-Song-Recommender-System-main/database/Meli.jpg', '/Users/vittoriomocchi/Documents/Vitos/Projects/Emotion-Song-Recommender-System-main/database/Mischa.jpg', '/Users/vittoriomocchi/Documents/Vitos/Projects/Emotion-Song-Recommender-System-main/database/Vito.jpg' ]  # replace with your image paths
    labels = ['Elena', 'Lena', 'Meli', 'Mischa', 'Vito']  # replace with your corresponding names
    
    for img_path, label in zip(images, labels):
        image = cv2.imread(img_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = detector(gray_image)

        if len(faces) > 0:
            face = faces[0]  # Use the first detected face
            shape = predictor(gray_image, face)
            face_encoding = np.array(face_rec_model.compute_face_descriptor(image, shape))

            # Add the face encoding and label to the lists
            known_face_encodings.append(face_encoding)
            known_face_labels.append(label)

# Recognize the face from an input image
def recognize_face(input_image_path):
    input_image = cv2.imread(input_image_path)
    gray_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray_image)

    if len(faces) > 0:
        face = faces[0]
        shape = predictor(gray_image, face)
        input_face_encoding = np.array(face_rec_model.compute_face_descriptor(input_image, shape))

        # Compare with known faces
        distances = np.linalg.norm(known_face_encodings - input_face_encoding, axis=1)
        min_distance_idx = np.argmin(distances)

        if distances[min_distance_idx] < 0.6:  # Threshold for recognition
            return known_face_labels[min_distance_idx]
        else:
            return "Unknown"
    return "No face detected"

# Load the known faces (your database)
load_known_faces()

# Now let's recognize a face from a new image
recognized_name = recognize_face('/Users/vittoriomocchi/Documents/Vitos/Projects/Emotion-Song-Recommender-System-main/IMG_5720.jpg')  # replace with the path to the image of you

print(f"Recognized as: {recognized_name}")


Recognized as: Vito


In [13]:
from deepface import DeepFace

# Detect emotions from the input image
def recognize_emotion(input_image_path):
    result = DeepFace.analyze(img_path=input_image_path, actions=['emotion'])
    return result[0]["dominant_emotion"]

# Detect emotions from the image
recognized_emotion = recognize_emotion('/Users/vittoriomocchi/Documents/Vitos/Projects/Emotion-Song-Recommender-System-main/IMG_5720.jpg')

print(f"Detected emotion: {recognized_emotion}")


Detected emotion: sad


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


## Connection to API Spotify 

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Set up authentication
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="ca5e1837d17442e1b5c842377539f4fb",
                                               client_secret="c97c23c09b7948b4b4f4025a08a15a39",
                                               redirect_uri="http://localhost:8888/callback",
                                               scope="user-top-read"))

# Get the current user's top tracks
top_tracks = sp.current_user_top_tracks(limit=20, offset=0, time_range='medium_term')

# Print the top tracks
for idx, track in enumerate(top_tracks['items']):
    print(f"{idx + 1}. {track['name']} by {', '.join(artist['name'] for artist in track['artists'])}")

1. Everyday Hustle by Future, Metro Boomin, Rick Ross
2. New Soul by Yael Naim
3. Dancing in the Moonlight by Toploader
4. Need U 2Nite (feat. Massimo Pericolo) by Guè, Massimo Pericolo
5. Angelina Jolie by Bresh, SHUNE
6. Suga Honey Iced Tea by Kelis
7. Afraid To Feel by LF SYSTEM
8. Con il nastro rosa by Lucio Battisti
9. Diventare Grande by Guè
10. Alright - Remastered 2013 by Jamiroquai
11. La Maritza by Sylvie Vartan
12. Nice For What by Drake
13. Dubbi non ho - 2017 Remaster by Pino Daniele
14. Senz E Me by Geolier
15. Beat It by Future, Metro Boomin
16. Rainfall (Praise You) by Tom Santa
17. My Oh My (feat. DaBaby) by Camila Cabello, DaBaby
18. Not Like Us by Kendrick Lamar
19. L'appuntamento by Ornella Vanoni
20. West Coast (feat. Blueface, ALLBLACK & YG) by G-Eazy, Blueface, ALLBLACK, YG


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Set up authentication with the additional scope
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="ca5e1837d17442e1b5c842377539f4fb",
                                               client_secret="c97c23c09b7948b4b4f4025a08a15a39",
                                               redirect_uri="http://localhost:8888/callback",
                                               scope="user-top-read playlist-read-private"))

# Get the current user's private playlists
playlists = sp.current_user_playlists()

# Print the playlists
for idx, playlist in enumerate(playlists['items']):
    print(f"{idx + 1}. {playlist['name']} - {playlist['tracks']['total']} tracks")

1. elenaperego + Solosoldi - 50 tracks
2. My recommendation playlist - 10 tracks
3. Reggaeton heroine - 15 tracks
4. Best Guitar Intros Ever - 119 tracks
5. Tatuz - 38 tracks
6. V - 64 tracks
7. non mi piacciono le voci femminili ma - 127 tracks
